In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import calendar
import glob
import datetime
import load_sdot_utils
import pandas as pd
from collections import defaultdict

In [9]:
curr_path = os.getcwd()
data_path = curr_path + '/../data'
raw_transaction_path = data_path + '/RawTransactionData'
belltown_path = data_path + '/Belltown_Minute'
commcore_path = data_path + '/CommercialCore_Minute'
pikepine_path = data_path + '/PikePine_Minute'
firsthill_path = data_path + '/FirstHill_Minute'
dennytriangle_path = data_path + '/DennyTriangle_Minute'

In [19]:
start_hour = 8
end_hour = 20
minute_interval = 60
months_years = [(3, 2016), (4, 2016), (5, 2016), (6, 2016), (7, 2016)]
file_paths = [belltown_path, firsthill_path, dennytriangle_path, commcore_path, pikepine_path]

In [20]:
load_sdot_utils.aggregate_loads(start_hour, end_hour, minute_interval, months_years, file_paths)